In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# Загрузим данные

Данных очень много, поэтому сразу отберём 10000 пользователей

In [2]:
user_data = pd.read_csv('data\\user_data\\user_data.csv',
                        dtype={'gender': 'int8', 'age': 'uint8', 'country': 'category', 'city': 'category',
                               'exp_group': 'int8', 'os': 'category', 'source': 'category'},
                        index_col='user_id')

print(f'Всего пользователей = {user_data.shape[0]}')

user_data = user_data.sample(10_000, random_state=42)

user_data.head()

Всего пользователей = 163205


,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
82908,0,35,Russia,Moscow,4,Android,ads
55992,1,21,Russia,Yalutorovsk,2,iOS,ads
289,0,45,Russia,Severodvinsk,0,Android,ads
125840,1,38,Russia,Vetluga,3,Android,organic
104581,0,22,Russia,Moscow,4,Android,ads


In [3]:
post_data = pd.read_csv('data\\post_data\\post_text_df.csv',
                        dtype={'text': 'string', 'topic': 'category'},
                        index_col='post_id')

print(f'Количество записей = {post_data.shape[0]}')
post_data.head()

Количество записей = 7023


,text,topic
post_id,,
1,UK economy facing major risks The UK manufact...,business
2,Aids and climate top Davos agenda Climate cha...,business
3,Asian quake hits European shares Shares in Eu...,business
4,India power shares jump on debut Shares in In...,business
5,Lacroix label bought by US firm Luxury goods ...,business


In [4]:
feed_data = pd.read_csv('data\\feed_data\\feed_data.csv',
                        dtype={'user_id': 'uint32', 'post_id': 'uint16', 'target': 'int8'},
                        parse_dates=['timestamp']
                       )

print(f'Было записей = {feed_data.shape[0]}')

feed_data = feed_data[feed_data['user_id'].isin(set(user_data.index))]

print(f'Стало записей = {feed_data.shape[0]}')

feed_data.head()

Было записей = 68686455
Стало записей = 4215797


,user_id,post_id,timestamp,target
15272,100142,5951,2021-10-02 07:47:36,0
15273,100142,5883,2021-10-02 07:48:39,0
15274,100142,1935,2021-10-02 07:50:20,0
15275,100142,3498,2021-10-02 07:53:19,0
15276,100142,4794,2021-10-02 07:55:39,0


# Обработаем данные

# user_data

Сначала посмотрим на распределение колонки `topic` в зависимости от параметров пользователя, сделаем это по коэффициенту корреляции между групп

Например, если коэффициенты корреляции между разными странами по среднему количеству лайков в разных топиках малы, значит страна пользователя не важна

In [5]:
for col in user_data.columns:
    feed_data[f'user_{col}'] = feed_data['user_id'].map(user_data[col])

feed_data['topic'] = feed_data['post_id'].map(post_data['topic'])

feed_data.sample(5)

,user_id,post_id,timestamp,target,user_gender,user_age,user_country,user_city,user_exp_group,user_os,user_source,topic
64783283,7584,4729,2021-12-01 12:04:23,0,1,38,Russia,Kazan,1,iOS,ads,movie
41342152,50093,1016,2021-12-10 09:11:39,0,0,21,Belarus,Drahichyn,4,iOS,ads,politics
52445644,12734,6804,2021-10-16 14:29:29,0,1,18,Russia,Taganrog,4,iOS,ads,movie
34785109,125557,2671,2021-10-31 17:41:25,0,1,20,Russia,Serov,0,Android,organic,covid
29850717,17664,1272,2021-12-23 21:02:04,0,0,16,Russia,Reutov,1,Android,ads,politics


In [6]:
print("Минимальный абсолютный коэффициент корреляции по колонке:")
for col in feed_data.columns:
    if col.startswith('user_') and col != 'user_id':
        print(f"{col} = {feed_data.pivot_table(index='topic', columns=col, values='target').corr().abs().min().min():.2f}")

Минимальный абсолютный коэффициент корреляции по колонке:
user_gender = 0.99
user_age = 0.00
user_country = 0.02
user_city = 0.00
user_exp_group = 0.96
user_os = 0.99
user_source = 0.99


В таблице `user_data` оставим колонки `age`, `country`, `city`

Последние 2 закодируем при помощи MTE

In [7]:
user_data = user_data[['age', 'country', 'city']]

user_data['city'] = user_data['city'].map(feed_data.groupby('user_city').target.mean())
user_data['country'] = user_data['country'].map(feed_data.groupby('user_country').target.mean())

user_data.head()

,age,country,city
user_id,,,
82908,35,0.112296,0.154482
55992,21,0.112296,0.101416
289,45,0.112296,0.130103
125840,38,0.112296,0.071006
104581,22,0.112296,0.154482


Добавим дополнительную информацию о пользователях, а именно процент лайков в каждой категории и в целом процент лайков, который будет показывать насколько пользователь "активен"

In [8]:
topic_pref = feed_data.pivot_table(index='user_id', columns='topic', values='target')
topic_pref.columns = [f"{col}_activity" for col in topic_pref.columns]

user_data = pd.concat([user_data, topic_pref], axis=1)

user_data['user_total_activity'] = feed_data.groupby('user_id').target.mean()

user_data.head()

,age,country,city,business_activity,covid_activity,entertainment_activity,movie_activity,politics_activity,sport_activity,tech_activity,user_total_activity
user_id,,,,,,,,,,,
82908,35,0.112296,0.154482,0.069767,0.194030,0.074074,0.230216,0.088496,0.101695,0.142857,0.147841
55992,21,0.112296,0.101416,0.166667,0.148148,0.076923,0.125714,0.129032,0.222222,0.090909,0.139706
289,45,0.112296,0.130103,0.086957,0.103093,0.000000,0.100000,0.074074,0.022222,0.150000,0.084615
125840,38,0.112296,0.071006,0.000000,0.073171,0.111111,0.084507,0.000000,0.000000,0.000000,0.056818
104581,22,0.112296,0.154482,0.090909,0.183673,0.200000,0.137931,0.272727,0.375000,0.000000,0.186747


In [9]:
print('Количество пропусков =')

user_data.isna().sum()

Количество пропусков =


age                       0
country                   0
city                      0
business_activity         0
covid_activity            0
entertainment_activity    4
movie_activity            0
politics_activity         0
sport_activity            0
tech_activity             2
user_total_activity       0
dtype: int64

Пропуски заполним средним значением по колонке и сохраним обработанный датасет

In [10]:
user_data.fillna(user_data.mean(), inplace=True)

user_data.to_csv('data\\user_data\\user_data_features.csv')

user_data.head()

C:\Users\dm200\AppData\Local\Temp\ipykernel_10912\1347893418.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_data.fillna(user_data.mean(), inplace=True)


,age,country,city,business_activity,covid_activity,entertainment_activity,movie_activity,politics_activity,sport_activity,tech_activity,user_total_activity
user_id,,,,,,,,,,,
82908,35,0.112296,0.154482,0.069767,0.194030,0.074074,0.230216,0.088496,0.101695,0.142857,0.147841
55992,21,0.112296,0.101416,0.166667,0.148148,0.076923,0.125714,0.129032,0.222222,0.090909,0.139706
289,45,0.112296,0.130103,0.086957,0.103093,0.000000,0.100000,0.074074,0.022222,0.150000,0.084615
125840,38,0.112296,0.071006,0.000000,0.073171,0.111111,0.084507,0.000000,0.000000,0.000000,0.056818
104581,22,0.112296,0.154482,0.090909,0.183673,0.200000,0.137931,0.272727,0.375000,0.000000,0.186747


# post_data

Добавим 2 колонки: количество просмотров, процент лайков

In [11]:
post_data['views'] = feed_data.groupby('post_id').target.count()
post_data['likes_percent'] = feed_data.groupby('post_id').target.mean()

post_data.head()

,text,topic,views,likes_percent
post_id,,,,
1,UK economy facing major risks The UK manufact...,business,484.0,0.128099
2,Aids and climate top Davos agenda Climate cha...,business,423.0,0.080378
3,Asian quake hits European shares Shares in Eu...,business,513.0,0.152047
4,India power shares jump on debut Shares in In...,business,522.0,0.139847
5,Lacroix label bought by US firm Luxury goods ...,business,513.0,0.120858


In [12]:
print('Количество пропусков =')

post_data.isna().sum()

Количество пропусков =


text               0
topic              0
views            192
likes_percent    192
dtype: int64

Это можно объяснить только тем, что не все посты представлены в таблице `feed_data`, поэтому удалим такие посты

In [13]:
post_data.dropna(inplace=True)

Колонку `views` приведём к целочисленному типу

In [14]:
post_data['views'] = post_data['views'].astype(int)

Сделаем MTE для колонки `topic`

In [15]:
post_data['topic'] = post_data['topic'].map(feed_data.groupby('topic').target.mean()).astype(float)

Обработаем текст

In [16]:
wnl = WordNetLemmatizer()

def preprocessing(line, token=wnl):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    line = line.replace('\n\n', ' ').replace('\n', ' ')
    line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
    return line


tfidf = TfidfVectorizer(stop_words='english', preprocessor=preprocessing)

tfidf_data = tfidf.fit_transform(post_data['text']).toarray()

tfidf_data = pd.DataFrame(tfidf_data,
                          index=post_data.index,
                          columns=tfidf.get_feature_names())

tfidf_data.head()

D:\Python_workspace\Lessons\DL\10_final_project\venv\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.
  warnings.warn(
D:\Python_workspace\Lessons\DL\10_final_project\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,0001,000bn,000m,000s,000th,001,001and,001st,...,𝓫𝓮,𝓫𝓮𝓽𝓽𝓮𝓻,𝓬𝓸𝓾𝓻𝓽𝓼,𝓱𝓮𝓪𝓻𝓲𝓷𝓰,𝓶𝓪𝔂,𝓹𝓱𝔂𝓼𝓲𝓬𝓪𝓵,𝓼𝓸𝓸𝓷𝓮𝓻,𝓼𝓾𝓫𝓸𝓻𝓭𝓲𝓷𝓪𝓽𝓮,𝓽𝓱𝓮,𝓽𝓸
post_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.135890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.051576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
n_components = 10

PCA_data = pd.DataFrame(
    data=PCA(n_components=n_components).fit_transform(tfidf_data - tfidf_data.mean()),
    index=post_data.index,
    columns=[f"PCA_{n}" for n in range(1, n_components + 1)])

PCA_data.head()

,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10
post_id,,,,,,,,,,
1,-0.087008,0.151647,-0.000607,-0.083132,-0.217895,0.079521,0.219510,0.002372,0.045834,-0.150027
2,-0.083853,0.194422,0.051568,0.037130,-0.058422,0.017237,-0.002660,-0.021204,0.016901,-0.006666
3,-0.063860,0.105556,-0.009263,-0.086243,-0.197605,0.020544,0.125654,0.016732,-0.000823,-0.011859
4,-0.076315,0.089829,0.001503,-0.048643,-0.147690,-0.000524,0.028847,0.025491,0.000988,0.065877
5,-0.059094,0.051467,-0.004680,-0.030744,-0.057165,-0.016601,-0.035494,-0.016148,-0.026589,0.039654


In [18]:
post_data_TFIDF = pd.concat([post_data.drop(columns='text'), PCA_data], axis=1)

post_data_TFIDF.to_csv('data\\post_data\\post_data_TFIDF_features.csv')

post_data_TFIDF.head()

,topic,views,likes_percent,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10
post_id,,,,,,,,,,,,,
1,0.119166,484,0.128099,-0.087008,0.151647,-0.000607,-0.083132,-0.217895,0.079521,0.219510,0.002372,0.045834,-0.150027
2,0.119166,423,0.080378,-0.083853,0.194422,0.051568,0.037130,-0.058422,0.017237,-0.002660,-0.021204,0.016901,-0.006666
3,0.119166,513,0.152047,-0.063860,0.105556,-0.009263,-0.086243,-0.197605,0.020544,0.125654,0.016732,-0.000823,-0.011859
4,0.119166,522,0.139847,-0.076315,0.089829,0.001503,-0.048643,-0.147690,-0.000524,0.028847,0.025491,0.000988,0.065877
5,0.119166,513,0.120858,-0.059094,0.051467,-0.004680,-0.030744,-0.057165,-0.016601,-0.035494,-0.016148,-0.026589,0.039654


# feed_data

Таблица была использована по максимуму, восстановим её к прежнему виду и обработаем колонку `timestamp`

Добавим следующие колонки:
1) Час дня
2) Дата в формате `yyyymmdd`

In [19]:
feed_data = feed_data[['user_id', 'post_id', 'timestamp', 'target']]

feed_data['hour'] = feed_data.timestamp.dt.hour
feed_data['data'] = feed_data.timestamp.map(lambda x: 10_000*x.year + 100*x.month + x.day)
feed_data.drop(columns='timestamp', inplace=True)

feed_data.to_csv('data\\feed_data\\feed_data_features.csv', index=False)

feed_data.sample(5)

,user_id,post_id,target,hour,data
48986661,12086,3649,0,16,20211004
38177409,56500,1949,0,7,20211016
49447438,33312,398,0,12,20211117
4856980,93902,3699,0,8,20211027
49607247,70609,5961,0,15,20211127


Проверим дисбаланс классов

In [20]:
feed_data['target'].value_counts() / len(feed_data)

0    0.880411
1    0.119589
Name: target, dtype: float64

Дисбаланс большой, учтём это при обучении моделей

А теперь обучать модель